In [ ]:
from math import log, sin, cos, tan, exp, sqrt, pi
from time import time, sleep
from random import randrange
import torch
import numpy as np
from testbed import ShortDataset, ByteDataset, Trainer, Net0, Net1, Net2, Net3, Net4, Transformer
from testbed.optimizer import Sonny
from testbed.util import default_device, numel
from testbed.gui import Plot, StatsTicker, ParameterInspector, Histogram, SmoothPlot, LinePlot

In [ ]:
network_name = "UTF8Transformer"

In [ ]:
if network_name == "Net0":
    num_input_classes= 256 # 256 possible UTF-8 bytes
    embedding_dim = 32 # Dimension of embedding space. An embedding layer has 256 points in this space.
    context_length = 32 # Number of sequential bytes visible to model (i.e. in the context)
    num_hidden = 8192 # Hyperparameter for neural network
    num_output_classes = 256 # 256 possible UTF-8 bytes
    model = Net0(num_input_classes=num_input_classes,
                 embedding_dim=embedding_dim,
                 context_length=context_length,
                 num_hidden=num_hidden,
                 num_output_classes=num_output_classes,
                 nonlinearity="sigmoid").to(default_device())
    example_length = context_length + 1
    dataset = ByteDataset(example_length=example_length)
    batch_size = 512 # batch size (i.e. examples per batch)
    OptimizerType = Sonny
    optimizer_kwargs = {"eps": 1e-8, "weight_decay": 0.01}

In [ ]:
utf8_transformer_config = {
    "model": {
        "type": Transformer,
        "kwargs": {
            "n_vocab": 256,
            "max_ctx": 128,
            "d_model": 1024,
            "n_heads": 8,
            "d_ff": 8192,
            "n_layers": 8}},
    "optimizer": {
        "type": Sonny,
        "kwargs": {
            "eps": 1e-4, 
            "lr": .0001, 
            "beta1": .9, 
            "beta2": .999,
            "weight_decay": 0.01,
            "warmup": 10000}},
    "dataset": {
        "type": ByteDataset,
        "kwargs": {
            "batch_size": 256,
            "example_length": 32+1}}}

In [ ]:
config = utf8_transformer_config

In [ ]:
trainer = Trainer(config=config)

In [ ]:
trainer.start()

In [ ]:
ticker = StatsTicker(trainer.metrics,  x='step', y='mean_loss', kind='line')
ticker

In [ ]:
print(''.join(list(trainer.autocomplete(n_generate=256, max_ctx=32))))

In [ ]:
t = time()
while time() - t < 1000:
    sleep(60)
    print(f"{int(time() - t)//60} minutes, step={trainer.metrics[-1]['step']},"
          f" mean_loss = {trainer.metrics[-1]['mean_loss']}")
    print(''.join(list(trainer.autocomplete(n_generate=256, max_ctx=32))))

In [ ]:
trainer.metrics[-1]["step"]

In [ ]:
StatsTicker(trainer.metrics, x='time', y='energy')

In [ ]:
trainer.update("optimizer", lr=.0001, beta1=.9, beta2=.999, batch_size=8, weight_decay=0.01)

In [ ]:
trainer.save(model.name())

In [ ]:
m = trainer.metrics[-1]
print(m, m["batch_losses"].shape)

In [ ]:
def floprecurse(d, s):
    results = []
    if type(d) == dict:
        try:
            if "time" in d and "energy" in d:
                results.append(f'"{s}": {{"time": {d["time"]}, "energy": {d["energy"]},'
                               f' "tflops"= {d["energy"]/d["time"]/1E12}}}')
        except:
            pass
        for (k, v) in d.items():
            results = results + floprecurse(v, f"{s}.{k}")
    if type(d) == list:
        for (k, v) in enumerate(d):
            results = results + floprecurse(v, f"{s}.{k}")
    return results            

In [ ]:
d = trainer.losses[-1]
d

In [ ]:
A = floprecurse(d, "root")
for a in A:
    print(a)

### SmoothPlot

In [ ]:
L = np.array([[x['step'],x['mean_loss']] for x in trainer.losses])
X = L[:,0]
Y = L[:,1]
def smoother(data, lag):
    cs = np.cumsum(data)
    return (cs[lag:] - cs[:-lag])/lag

class SmoothPlot(LinePlot):
    def __init__(self, X=None, Y=None, lag=100, log=None):
        if X is not None:
            if Y is None:
                Y = np.array(X)
                X = np.array(range(len(X)))
            else:
                X = np.array(X)
                Y = np.array(Y)
            X = X[lag:]
            Y = smoother(Y, lag)
            if log:
                X = np.log(X)/math.log(2)
        super().__init__(X, Y)
SmoothPlot(X, Y, lag=100, log=False)

# Testing

In [ ]:
from time import time
def timing_study():

    n_vocab=50257
    max_ctx=1024
    d_model=1024
    n_heads=32
    d_ff=4096,
    n_layers=4
    n_ctx = 1024
    d_head = d_model // n_heads
    assert d_model == n_heads * d_head
    torch.cuda.synchronize()
    data = defaultdict(lambda: defaultdict(float))
    X = torch.randn(n_ctx, d_model, device='cuda')
    Q = X
    K = X
    V = X
    input_shape = X.shape
    def split_heads(x):
        return x.view(x.shape[:-1] + (n_heads, d_head)).transpose(-2, -3).contiguous()
    def merge_heads(x):
        x = x.transpose(-2,-3).contiguous()
        return x.view(x.shape[:-2] + (d_model,))
    additive_mask = 1.0-1.0/torch.tril(torch.ones(n_ctx,n_ctx, device=X.device))
    additive_mask = torch.zeros(n_ctx,n_ctx, device=X.device)
    @torch.no_grad()
    def compute(Q0,K0,V0,data, N):
        torch.cuda.synchronize()
        t = time()
        for _ in range(N):
            Q = split_heads(Q0)/math.sqrt(d_head)
            K = split_heads(K0)
            V = split_heads(V0)
            torch.cuda.synchronize()
            data["split_heads"]["data"] += torch.numel(Q) + torch.numel(K) + torch.numel(V)
        data["split_heads"]["energy"] += 3.0 * N * (torch.numel(Q) + torch.numel(K) + torch.numel(V))
        data["split_heads"]["time"] += time() - t


        t = time()
        for _ in range(N):
            QKT = torch.matmul(Q, K.transpose(-1,-2)).contiguous()
            torch.cuda.synchronize()
            data["matmul(Q,K^T)"]["data"] += torch.numel(Q) + torch.numel(K) + torch.numel(QKT)
        data["matmul(Q,K^T)"]["energy"] += 3.0 * N * torch.numel(Q) * n_ctx
        data["matmul(Q,K^T)"]["time"] += time() - t

        #print(f"Q.shape={Q.shape}")
        #print(f"K.shape={Q.shape}")
        #print(f"QKT.shape={QKT.shape}")
        t = time()
        for _ in range(N):
            Z = QKT + additive_mask
        torch.cuda.synchronize()
        data["masking_attention"]["data"] += N*(torch.numel(Z) + torch.numel(QKT) + torch.numel(additive_mask))
        data["masking_attention"]["energy"] += 3.0 * N * 4.0 * (torch.numel(Z)) # ?
        data["masking_attention"]["time"] += time() - t
    

#         t = time()
#         for _ in range(N):
#             (Zmax, _) = torch.max(Z,dim=-1,keepdim=True)
#             Z = Z - Zmax # as in GPT-2
#             EZ = torch.exp(Z)
#             sumEZ = torch.sum(EZ,dim=-1,keepdim=True)
#             A = EZ/sumEZ
#             assert A.shape == EZ.shape
#             torch.cuda.synchronize()
#         data["compute_attention"]["energy"] += 3.0 * N * 5.0 * (torch.numel(A)) # ?
#         data["compute_attention"]["time"] += time() - t
        softmax = torch.nn.Softmax(dim=-1)
        t = time()
        for _ in range(N):
            A = softmax(Z)
            torch.cuda.synchronize()
            data["compute_attention"]["data"] += torch.numel(Z) + torch.numel(A)
        data["compute_attention"]["energy"] += 3.0 * N * 5.0 * (torch.numel(A)) # ?
        data["compute_attention"]["time"] += time() - t
        
        t = time()
        for _ in range(N):
            AV = torch.matmul(A,V)
            torch.cuda.synchronize()
            data["matmul(A,V)"]["data"] += torch.numel(A) + torch.numel(V) + torch.numel(AV)
        data["matmul(A,V)"]["energy"] += 3.0 * N * (torch.numel(A) * d_head)
        data["matmul(A,V)"]["time"] += time() - t

        t = time()
        for _ in range(N):
            mergedAV = merge_heads(AV)
            torch.cuda.synchronize()
            data["merge_heads"]["data"] += torch.numel(AV)
        data["merge_heads"]["energy"] += 3.0 * N * (torch.numel(AV))
        data["merge_heads"]["time"] += time() - t
    start_time = time()
    idx = 0
    while time() < start_time + 30:
        compute(Q,K,V,data,64)
        idx = idx + 1
    return (idx, data)

In [ ]:
idx, data = timing_study()

In [ ]:
idx

In [ ]:
data

In [ ]:
ttime = sum(v["time"] for (k,v) in data.items())
tenergy = sum(v["energy"] for (k,v) in data.items())

for (k,v) in data.items():
    print(k, v["time"]/ttime, v["energy"]/v["time"]/1E12, v["data"]/v["time"]/1E9)

In [ ]:
tenergy/ttime/1E12

In [ ]:
s = torch.nn.Softmax(dim=-1)
x = 1.0-1.0/torch.tril(torch.ones(5,5, device='cpu'))
s(x)